## The Python SQL Toolkit and Object Relational Mapper

SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

It provides a full suite of well known enterprise-level persistence patterns, designed for efficient and high-performing database access, adapted into a simple and Pythonic domain language.

SQLAlchemy is used by http://www.sqlalchemy.org/organizations.html

SQLAlchemy installation

```
pip install sqlalchemy
```

**Object-relational Mapping** (ORM, O/RM, and O/R mapping) in computer software is a programming technique for converting data between incompatible type systems in relational databases and object-oriented programming languages. This creates, in effect, a "virtual object database" that can be used from within the programming language. There are both free and commercial packages available that perform object-relational mapping, although some programmers opt to create their own ORM tools.

### SQLAlchemy and Object-Relational Mapping

A common task when programming any web service is the construction of a solid database backend. In the past, programmers would write raw SQL statements, pass them to the database engine and parse the returned results as a normal array of records. Nowadays, programmers can write Object-relational mapping (ORM) programs to remove the necessity of writing tedious and error-prone raw SQL statements that are inflexible and hard-to-maintain.

ORM is a programming technique for converting data between incompatible type systems in object-oriented programming languages. Usually, the type system used in an OO language such as Python contains types that are non-scalar, namely that those types cannot be expressed as primitive types such as integers and strings. For example, a Person object may have a list of Address objects and a list of PhoneNumber objects associated with it. In turn, an Address object may have a PostCode object, a StreetName object and a StreetNumber object associated with it. Although simple objects such as PostCodes and StreetNames can be expressed as strings, a complex object such as a Address and a Person cannot be expressed using only strings or integers. In addition, these complex objects may also include instance or class methods that cannot be expressed using a type at all.

In order to deal with the complexity of managing objects, people developed a new class of systems called ORM. Our previous example can be expressed as an ORM system with a Person class, a Address class and a PhoneNumber class, where each class maps to a table in the underlying database. Instead of writing tedious database interfacing code yourself, an ORM takes care of these issues for you while you can focus on programming the logics of the system.

![Old vs New DB](oldvsnew.png)

### Old Way

```sqllite3``` database connector is included by default in python. Here we are instantiating a connector to a database called example.db

In [1]:
import sqlite3
conn = sqlite3.connect('example.db')
c = conn.cursor()

Create a table ```person``` with ```id``` and ```name```

In [2]:
c.execute('''
          CREATE TABLE person
          (id INTEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)
          ''')

In [3]:
c.execute('''
          CREATE TABLE IF NOT EXISTS person
          (id INTEGER PRIMARY KEY ASC, name varchar(250) NOT NULL)
          ''')

Create a relational table ```address``` with ```street_name, street_number, post_code``` with foreign key relationship on ```person_id```

In [4]:
c.execute('''
          CREATE TABLE IF NOT EXISTS address
          (id INTEGER PRIMARY KEY ASC, street_name varchar(250), street_number varchar(250),
           post_code varchar(250) NOT NULL, person_id INTEGER NOT NULL,
           FOREIGN KEY(person_id) REFERENCES person(id))
          ''')

Insert some default value commit and close the connection

In [5]:
c.execute('''
          INSERT INTO person VALUES(1, 'pythoncentral')
          ''')
c.execute('''
          INSERT INTO address VALUES(1, 'python road', '1', '00000', 1)
          ''')
 
conn.commit()
conn.close()

Query the database

In [6]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

In [7]:
c.execute('SELECT * FROM person')
print c.fetchall()

[(1, u'pythoncentral')]


In [8]:
c.execute('SELECT * FROM address')
print c.fetchall()

[(1, u'python road', u'1', u'00000', 1)]


### SQLAlchemy way

In [9]:
import sqlalchemy
sqlalchemy.__version__ 

'1.0.8'

To connect use ```create_engine```

In [10]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///person2.db', echo=False)

**Declare a Mapping** 

When using the ORM, the configurational process starts by describing the database tables we’ll be dealing with, and then by defining our own classes which will be mapped to those tables. In modern SQLAlchemy, these two tasks are usually performed together, using a system known as Declarative, which allows us to create classes that include directives to describe the actual database table they will be mapped to.

Classes mapped using the Declarative system are defined in terms of a base class which maintains a catalog of classes and tables relative to that base - this is known as the declarative base class. Our application will usually have just one instance of this base in a commonly imported module. We create the base class using the declarative_base() function, as follows:

In [11]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [12]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

A class using Declarative at a minimum needs a ```__tablename__``` attribute, and at least one Column which is part of a primary key

With our User class constructed via the Declarative system, we have defined information about our table, known as table metadata. The object used by SQLAlchemy to represent this information for a specific table is called the Table object, and here Declarative has made one for us. We can see this object by inspecting the ```__table__``` attribute:

In [13]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

To create the actual table

In [14]:
Base.metadata.create_all(engine) 

To create a User object

In [15]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [16]:
ed_user.name

'ed'

In [17]:
ed_user.password

'edspassword'

In [18]:
str(ed_user.id)

'None'

#### Creating a Session

We’re now ready to start talking to the database. The ORM’s “handle” to the database is the `Session`. When we first set up the application, at the same level as our `create_engine()` statement, we define a Session class which will serve as a factory for new `Session` objects:

In [19]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

To persist our User object, `we add()` it to our `Session`

In [20]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)

At this point, we say that the instance is **pending**; no SQL has yet been issued and the object is not yet represented by a row in the database. The `Session` will issue the SQL to persist Ed Jones as soon as is needed, using a process known as a **flush**. If we query the database for Ed Jones, all pending information will first be flushed, and the query is issued immediately thereafter.

In [21]:
our_user = session.query(User)

In [22]:
for each in our_user:
    print each

<User(name='ed', fullname='Ed Jones', password='edspassword')>


In [23]:
for instance in session.query(User).order_by(User.id): 
    print instance.name, instance.fullname

ed Ed Jones


We can add more User objects at once using `add_all()`:

In [24]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

In [25]:
session.new 

IdentitySet([<User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>])

In [26]:
ed_user.password = 'f8s7ccs'

The `Session` is paying attention. It knows, for example, that `Ed Jones` has been modified:

In [27]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>])

In [28]:
from sqlalchemy import Table, MetaData
meta = MetaData(bind=engine, reflect=True)
table = meta.tables['users']
list(engine.execute(table.select()))

/home/dg/Documents/MSU/big-data/big-data-env/lib/python2.7/site-packages/ipykernel/__main__.py:2: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  from ipykernel import kernelapp as app


[]

We tell the `Session` that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via `commit()`:

In [29]:
session.commit()

In [30]:
str(ed_user.id)

'1'

#### Rolling Back
Since the `Session` works within a transaction, we can roll back changes made too. Let’s make two changes that we’ll revert; ed_user‘s user name gets set to `Edwardo`:

In [31]:
ed_user.name = 'Edwardo'

In [32]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

In [33]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

Rolling back, we can see that ed_user‘s name is back to ed, and fake_user has been kicked out of the session:

In [34]:
session.rollback()

In [35]:
ed_user.name 

u'ed'

In [36]:
fake_user in session

False

#### Querying

A Query object is created using the query() method on Session. This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors. Below, we indicate a Query which loads User instances. When evaluated in an iterative context, the list of User objects present is returned:

In [37]:
for instance in session.query(User).order_by(User.id): 
    print instance.name, instance.fullname

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [38]:
for name, fullname in session.query(User.name, User.fullname): 
    print name, fullname

ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [39]:
for row in session.query(User, User.name).all(): 
    print row.User, row.name

<User(name='ed', fullname='Ed Jones', password='f8s7ccs')> ed
<User(name='wendy', fullname='Wendy Williams', password='foobar')> wendy
<User(name='mary', fullname='Mary Contrary', password='xxg527')> mary
<User(name='fred', fullname='Fred Flinstone', password='blah')> fred


In [40]:
for row in session.query(User.name.label('name_label')).all(): 
    print(row.name_label)

ed
wendy
mary
fred


In [41]:
for u in session.query(User).order_by(User.id)[1:3]: 
    print u

<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>


In [42]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'): 
    print name

ed


#### Common Filter Operators

Here’s a rundown of some of the most common operators used in filter():

![Query Types](query.png)
http://docs.sqlalchemy.org/en/rel_0_9/orm/tutorial.html#common-filter-operators

#### Working with Related Objects

Let’s consider how a second table, related to `User`, can be mapped and queried. Users in our system can store any number of email addresses associated with their username. This implies a basic one to many association from the users to a new table which stores email addresses, which we will call addresses. Using declarative, we define this table along with its mapped class, `Address`:

In [43]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship("User", backref=backref('addresses', order_by=id))
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address



In [44]:
Base.metadata.create_all(engine) 

In [45]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses

[]

In [46]:
jack.addresses = [
    Address(email_address='jack@google.com'),
    Address(email_address='j25@yahoo.com')]

In [47]:
jack.addresses[1]

<Address(email_address='j25@yahoo.com')>

In [48]:
jack.addresses[1].user

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

In [49]:
session.add(jack)
session.commit()

In [50]:
jack = session.query(User).filter_by(name='jack').one()
jack

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

In [51]:
jack.addresses 

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

#### Querying with Joins

Now that we have two tables, we can show some more features of Query, specifically how to create queries that deal with both tables at the same time. The Wikipedia page on SQL JOIN offers a good introduction to join techniques, several of which we’ll illustrate here.

To construct a simple implicit join between User and Address, we can use Query.filter() to equate their related columns together. Below we load the User and Address entities at once using this method:

In [52]:
for u, a in session.query(User, Address).\
                        filter(User.id==Address.user_id).\
                        filter(Address.email_address=='jack@google.com').\
                        all():   
    print u
    print a

<User(name='jack', fullname='Jack Bean', password='gjffdd')>
<Address(email_address='jack@google.com')>


The actual SQL JOIN syntax, on the other hand, is most easily achieved using the `Query.join()` method:

In [53]:
session.query(User).join(Address).\
         filter(Address.email_address=='jack@google.com').\
         all() 

[<User(name='jack', fullname='Jack Bean', password='gjffdd')>]

#### Eager Loading
If you want to reduce the number of queries (dramatically, in many cases), we can apply an eager load to the query operation. SQLAlchemy offers three types of eager loading, two of which are automatic, and a third which involves custom criterion. All three are usually invoked via functions known as query options which give additional instructions to the `Query` on how we would like various attributes to be loaded, via the `Query.options()` method.

* **Subquery Load:**  In this case we’d like to indicate that `User.addresses` should load eagerly. A good choice for loading a set of objects as well as their related collections is the `orm.subqueryload()` option, which emits a second SELECT statement that fully loads the collections associated with the results just loaded. The name “subquery” originates from the fact that the `SELECT` statement constructed directly via the `Query` is re-used, embedded as a subquery into a `SELECT` against the related table. This is a little elaborate but very easy to use:

In [54]:
from sqlalchemy.orm import subqueryload
jack = session.query(User).\
                 options(subqueryload(User.addresses)).\
                 filter_by(name='jack').one() 

In [55]:
jack

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

In [56]:
jack.addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

* **Joined Load**: The other automatic eager loading function is more well known and is called `orm.joinedload()`. This style of loading emits a `JOIN`, by default a `LEFT OUTER JOIN`, so that the lead object as well as the related object or collection is loaded in one step. We illustrate loading the same addresses collection in this way - note that even though the User.addresses collection on jack is actually populated right now, the query will emit the extra join regardless:

In [57]:
from sqlalchemy.orm import joinedload
jack = session.query(User).\
                        options(joinedload(User.addresses)).\
                        filter_by(name='jack').one() 

In [58]:
jack

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

In [59]:
jack.addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

Note that even though the `OUTER JOIN` resulted in two rows, we still only got one instance of User back. This is because Query applies a “uniquing” strategy, based on object identity, to the returned entities. This is specifically so that joined eager loading can be applied without affecting the query results.

While `joinedload()` has been around for a long time, `subqueryload()` is a newer form of eager loading. `subqueryload()` tends to be more appropriate for loading related collections while `joinedload()` tends to be better suited for many-to-one relationships, due to the fact that only one row is loaded for both the lead and the related object.

* **Explicit Join + Eagerload**: A third style of eager loading is when we are constructing a `JOIN` explicitly in order to locate the primary rows, and would like to additionally apply the extra table to a related object or collection on the primary object. This feature is supplied via the `orm.contains_eager()` function, and is most typically useful for pre-loading the many-to-one object on a query that needs to filter on that same object. Below we illustrate loading an Address row as well as the related User object, filtering on the User named “jack” and using `orm.contains_eager()` to apply the “user” columns to the `Address.user` attribute:

In [60]:
from sqlalchemy.orm import contains_eager

jacks_addresses = session.query(Address).\
                             join(Address.user).\
                             filter(User.name=='jack').\
                             options(contains_eager(Address.user)).\
                             all()

In [61]:
jacks_addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [62]:
jacks_addresses[0].user

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

#### Deleting

Let’s try to delete jack and see how that goes. We’ll mark as deleted in the session, then we’ll issue a count query to see that no rows remain:

In [63]:
session.delete(jack)

In [64]:
session.query(User).filter_by(name='jack').count() 

0

So far, so good. How about Jack’s `Address` objects ?

In [65]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

2

##### Configuring delete/delete-orphan Cascade

We will configure cascade options on the `User.addresses` relationship to change the behavior. While SQLAlchemy allows you to add new attributes and relationships to mappings at any point in time, in this case the existing relationship needs to be removed, so we need to tear down the mappings completely and start again - we’ll close the `Session`:

In [66]:
session.close()
Base = declarative_base()

In [67]:
class User(Base):
     __tablename__ = 'users'

     id = Column(Integer, primary_key=True)
     name = Column(String)
     fullname = Column(String)
     password = Column(String)

     addresses = relationship("Address", backref='user',
                     cascade="all, delete, delete-orphan")

     def __repr__(self):
        return "<User(name='%s', fullname='%s', password'%s')>" % (
                                self.name, self.fullname, self.password)

In [68]:
class Address(Base):
     __tablename__ = 'addresses'
     id = Column(Integer, primary_key=True)
     email_address = Column(String, nullable=False)
     user_id = Column(Integer, ForeignKey('users.id'))

     def __repr__(self):
         return "<Address(email_address='%s')>" % self.email_address

Now when we load the user jack (below using `get()`, which loads by primary key), removing an address from the corresponding addresses collection will result in that `Address` being deleted:

load Jack by primary key

In [69]:
jack = session.query(User).get(5) 

remove one Address (lazy load fires off)

In [70]:
del jack.addresses[1] 

only one address remains

In [71]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count() 

1

Deleting Jack will delete both Jack and the remaining `Address` associated with the user:

In [72]:
session.delete(jack)

In [73]:
session.query(User).filter_by(name='jack').count()

0

In [74]:
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count() 

0